In [ ]:
#import basic modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
import time
import seaborn as sns

warnings.filterwarnings("ignore") 
pd.options.display.width = 500


# Exploratory Data Analysis
## 1. Getting the data

In [ ]:
winter_data = pd.read_csv(
    'IndividualClothingValue.csv', index_col=0)


In [ ]:
winter_data.shape

In [ ]:
winter_data.head()

In [ ]:
winter_data.describe()

In [ ]:
winter_data.info()

In [ ]:
winter_data.columns

In [ ]:
winter_data = winter_data.astype({
    'DAY':'category',
    'School':'category',
    'SchoolType': 'category',
    'StartTime': 'category',
    'Grade': 'category',
    'Gender': 'category',
    'FormalClothing': 'bool',
    'Pant': 'bool',
    'Trackpant': 'bool',
    'Halfshirt': 'bool',
    'Blazer': 'bool',
    'Jacket': 'bool',
    'Skirt': 'bool',
    'FullShirt': 'bool',
    'HalfSweater': 'bool',
    'Tshirt': 'bool',
    'Socks': 'bool',
    'Thermal': 'bool',
    'Vest': 'bool',
    'FullSweater': 'bool',
    'TSV':'category',
    'TPV':'category',
    'TCV':'category',
    'TSL':'category',
    'MC':'category',
    'SwC':'category',
})

In [ ]:
winter_data.info()

## Distributions of Numerical Features

In [ ]:
# create a new dataframe with columns containing only the numerical features
num_features = winter_data.select_dtypes(exclude=['bool', 'category']).copy()

# we plot individual column distributions with null entry rows dropped
fig, axs = plt.subplots(2, 5, figsize=(10, 6))
for i in range(len(num_features.columns)):
    plt.subplot(2, 5, i+1)
    sns.distplot(num_features.iloc[:,i].dropna())
    plt.xlabel(num_features.columns[i])

plt.tight_layout()
plt.show()

## Distributions of Categorical Features

In [ ]:
# create a new dataframe with columns containing only the categorical features
cat_features = winter_data.select_dtypes(include=['category']).copy()

# we plot individual column distributions with null entry rows dropped
fig, axs = plt.subplots(nrows=7, ncols=2, figsize=(10, 30))
plt.subplots_adjust(right=1.5, top=1.25)

for i in range(len(cat_features.columns)):
    plt.subplot(7, 2, i+1)
    sns.countplot(y=cat_features.columns[i], data=cat_features)
    
plt.tight_layout()
plt.show()

## Distributions of Boolean Features

In [ ]:
# create a new dataframe with columns containing only the categorical features
bool_features = winter_data.select_dtypes(include='bool').copy()

# we plot individual column distributions with null entry rows dropped
fig, axs = plt.subplots(nrows=7, ncols=2, figsize=(10, 30))
plt.subplots_adjust(right=1.5, top=1.25)

for i in range(len(bool_features.columns)):
    plt.subplot(7, 2, i+1)
    sns.countplot(y=bool_features.columns[i], data=bool_features)
    
plt.tight_layout()
plt.show()

In [ ]:
tsl_target_data = winter_data.copy()

# Machine Learning Models (scikit-learn)

In [ ]:
# Import classifiers
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier

# Function for splitting training and test set
from sklearn.model_selection import train_test_split

# Function for creating model pipelines
from sklearn.pipeline import  make_pipeline

# Helper for cross-validation
from sklearn.model_selection import GridSearchCV, GroupKFold, StratifiedKFold

# For standardization
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler

# For dimensionality reduction
from sklearn.decomposition import PCA



In [ ]:
# Drop samples where target has samples < 2.
clean_winter_data = winter_data.copy()[winter_data.TPV != -1][winter_data.TSV != 2][winter_data.TCV != -3]
clean_winter_data.shape

In [ ]:
sklearn_data = pd.get_dummies(clean_winter_data.copy().drop(['TSV', 'TPV',
       'TSL','TCV'], axis=1), cat_features.drop(['TSV', 'TPV',
       'TSL','TCV'], axis=1).columns)
sklearn_data.head()

In [ ]:
sklearn_data.info()

## **1. Split the data**

In [ ]:
# Define our features dataframe X and labels y
X= sklearn_data
y= clean_winter_data.TSV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=505, stratify=y)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))



In [ ]:
# Pipeline dictionary
pipelines = {
    'lr': make_pipeline(StandardScaler(), 
                        LogisticRegressionCV(n_jobs=-1, random_state=505)),
   # 'rdg': make_pipeline(StandardScaler(),
    #                       RidgeClassifierCV()),
    'svc': make_pipeline(StandardScaler(),
                        SVC(random_state=505, probability=True)),
    'rf': make_pipeline(StandardScaler(), 
                            RandomForestClassifier(n_estimators=100, max_depth=15, random_state=505)),
    #Kneighbours
    'knc': make_pipeline(StandardScaler(),
                            KNeighborsClassifier(n_jobs=-1)),
    #gausian process
    'gpc': make_pipeline(StandardScaler(),
                            GaussianProcessClassifier(n_jobs=-1,random_state=505)),
}



## Choice of metric evaluation

1. Quadratic Cohen's Kappa

    Cohen’s kappa statistic measures interrater reliability (sometimes called interobserver agreement). Interrater reliability, or precision, happens when your data raters (or collectors) give the same score to the same data item. In our case, we have labels VS predictions. The Kappa statistic takes into account this element of chance.

The Kappa statistic varies from 0 to 1, where.

0 = agreement equivalent to chance.\ 0.1 – 0.20 = slight agreement.\ 0.21 – 0.40 = fair agreement.\ 0.41 – 0.60 = moderate agreement.\ 0.61 – 0.80 = substantial agreement.\ 0.81 – 0.99 = near perfect agreement\ 1 = perfect agreement.

2. Micro-averaged Precision score

    The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

For multiclass classification, Micro-average is preferable if there is a class imbalance problem.

In [ ]:
# Classification metrics 
from sklearn.metrics import precision_score, accuracy_score, cohen_kappa_score

# Initiate empty dictionary of fitted base models
fitted_basemodels = {}
# Loop through model pipelines
for name, pipeline in pipelines.items():
  
    # Fit model on X_train, y_train  and predict with X_test
    base_mod = pipeline.fit(X_train, y_train)
    #pred = pipeline.predict(X_test)
    pred = pipeline.predict(X_test)
    pred_prob = pipeline.predict_proba(X_test)
    
    # Store model in fitted_models[name] 
    fitted_basemodels[name] = base_mod
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')
   # print('accuracy on test: ', pipeline.score(X_test,y_test))
    print('precision scores: ', precision_score(y_test, pred,  average='micro'))
    print('Accuracy scores ', accuracy_score(y_test, pred))
    print('Kappa on test: ', cohen_kappa_score(y_test, pred, weights='quadratic'))
    print('\n')  

**3. Hyperparameter tuning (Optional)**

In [ ]:
pipelines['rf'].get_params() # This is how we get the format of hyperparameter dictionaries for GridSearch

In [ ]:
# Create a dictionary for each models hyperparameters

svc_hyparams = {'svc__C': [0.1, 1, 10, 100, 1000],
               'svc__gamma': [0.1, 1, 10, 100]
               }

rf_hyparams  = {
    'randomforestclassifier__n_estimators': [100,200],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33],
    'randomforestclassifier__max_depth': [8, 15],
    'randomforestclassifier__min_samples_split': [2, 5 ],
    'randomforestclassifier__min_samples_leaf': [1, 2, 10]
}

knc_hyparams =  {
    'kneighborsclassifier__weights': ['uniform', 'distance'],
    'kneighborsclassifier__p': [1, 1.5 , 2, 10],
    'kneighborsclassifier__n_neighbors': np.arange(2,10),
 }



In [ ]:
# Now create a dictionary of hyperparameter dictionaries !?
hyperparameters = {
    'lr': {},
    'svc': svc_hyparams,
    'rf': rf_hyparams,
    'knc': knc_hyparams,
    'gpc':{}
}



In [ ]:


# Helper for cross-validation
from sklearn.model_selection import GridSearchCV

# Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv = 5, n_jobs=-1)
    
    # Fit model on X_train, y_train
    model.fit(X_train, y_train)
    
    # Store model in fitted_models[name] 
    fitted_models[name] = model
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')



In [ ]:
import pickle
for name, model in fitted_models.items():
    pickle.dump(model, open(name+'.sav', 'wb'))

In [ ]:
fitted_models['lr']

In [ ]:


for name, model in fitted_models.items():
    
    pred = model.predict(X_test)
    print('precision score: ', precision_score(y_test, pred,  average='micro'))
    print('Accuracy scores ', accuracy_score(y_test, pred))
    print(name,'Kappa on test:', cohen_kappa_score(y_test, pred, weights='quadratic'))
    print('-'*30)



In [ ]:


feature_names = np.r_[X_train.columns.to_list()]
tree_feature_importances = (
    fitted_models['rf'].estimator['randomforestclassifier'].feature_importances_.reshape(1,len(feature_names))[0])
sorted_idx = tree_feature_importances.argsort()
print(feature_names[sorted_idx])
print(tree_feature_importances[sorted_idx])

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize=(8,8))
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()



In [ ]:
import dice_ml

In [ ]:
d = dice_ml.Data(dataframe=X_train, continuous_features=['AvgMaxDailyTemp', 'AvgMinDailyTemp',
                                                        'AvgIndoorRelativeHumidity', 'IndoorTempDuringSurvey'],
                                                        outcome_name='Age')

In [ ]:
m = dice_ml.Model(model=fitted_models['lr'], backend='sklearn')

In [ ]:
exp = dice_ml.Dice(d,m)

In [ ]:
query_instance = X_test[0:1].drop('Age', axis=1)

In [ ]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

## Machine Learning Models (CatBoost)

In [ ]:
import catboost as cat
# Classification metrics 
from sklearn.metrics import precision_score, cohen_kappa_score, f1_score, accuracy_score, recall_score
# Helper for cross-validation
from sklearn.model_selection import StratifiedKFold

In [ ]:
def oof_trainer(X: pd.DataFrame,
                y,
            n_folds = None,
            params: dict = None,
            del_cols: list = None,
            cat_features=None):
    
    """This function trains multiple Catboost model while performing stratified CV with shuffling.
    out-of-fold (oof) predictions are evaluated at each fold and printed out 
    at the end of the routine as a list. The mean scores on all fold is also printed out.
    
    The metrics used for evaluation are precision_auc and the cohen's kappa
    
    Output: 
    models -- a list of models trained on each fold during CV
    oof_pred -- prediction array consisting of predictions coming from different models
    """
        
    # collect models and scores from each fold
    models = []
    f1_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    scores = []
    # CV splitter
    folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=505) #GroupKFold(n_splits=n_folds) 
    #features to use
    columns = [col for col in X.columns.values if not col in del_cols]
    # dimension of the target 
    n_target=1
    
    # collect out-of-sample predictions
    data_X, data_y, oof_pred = pd.DataFrame(), pd.Series(), np.zeros((len(X),n_target))


    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
     
        print('Fold {} started at {}'.format(fold_n + 1,time.ctime()))
        #print((train_index, valid_index))
        X_train, X_valid = X.iloc[train_index][columns], X.iloc[valid_index][columns]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
        data_X = data_X.append(X_valid)
        data_y = data_y.append(y_valid)
        print(data_X.shape)
        
        #Eval set preparation
        eval_set = [(X_train, y_train)]
       # eval_names = ['train']
        eval_set.append((X_valid, y_valid))
       # eval_names.append('valid')
    
        model = cat.CatBoostClassifier()
        model.fit(X=X_train, y=y_train, 
                       eval_set=eval_set, 
                       verbose=500, early_stopping_rounds=150,
                      cat_features=cat_features, use_best_model=True, plot=True)
        
        oof_pred[valid_index] = model.predict(X_valid).reshape(-1, n_target)
        models.append(model)
    
        print('-'*30)
        
        scores.append(cohen_kappa_score(y_valid, oof_pred[valid_index], weights='quadratic'))
        f1_scores.append(f1_score(y_valid, oof_pred[valid_index],   average=None))
        accuracy_scores.append(accuracy_score(y_valid, oof_pred[valid_index]))
        precision_scores.append(precision_score(y_valid, oof_pred[valid_index],   average=None))
        recall_scores.append(recall_score(y_valid, oof_pred[valid_index],   average=None))

    print(f'catb fold kappa scores: {scores}')
    print(f'catb fold kappa scores mean: {np.mean(scores)}')
    print(f'catb fold f1 scores: {f1_scores}')
    print(f'catb fold f1 scores mean: {np.mean(f1_scores)}')
    print(f'catb fold precision-micro scores: {precision_scores}')
    print(f'catb fold precision-micro scores mean: {np.mean(precision_scores)}')
    print(f'catb fold recall-micro scores: {recall_scores}')
    print(f'catb fold recall-micro scores mean: {np.mean(recall_scores)}')
    print(f'catb fold accuracy scores: {accuracy_scores}')
    print(f'catb fold accuracy scores mean: {np.mean(accuracy_scores)}')
   
    return models, oof_pred


# Target: TSL

In [ ]:
# feature selection for Catboost
models, oof_pred = oof_trainer(X=tsl_target_data,
                    y=tsl_target_data.TSL,
                    n_folds = 5,
                   # params=params,
                    del_cols=['TSV',
       'TPV','TCV', 'TSL'],
                    cat_features=cat_features.drop(columns=['TSV',
       'TPV','TCV', 'TSL']).columns.values.tolist())

## Feature importance

In [ ]:
best_gb_model = models[1]
best_gb_model.get_feature_importance(prettified=True)

# Target: TPV

In [ ]:
# feature selection for Catboost
# Remove all rows with TPV value -1
models, oof_pred = oof_trainer(X=tsl_target_data[tsl_target_data.TPV != -1],
                    y=tsl_target_data[tsl_target_data.TPV != -1].TPV,
                    n_folds = 5,
                   # params=params,
                    del_cols=['TSV',
       'TSL','TCV', 'TSL', 'TPV'],
                    cat_features=cat_features.drop(columns=['TSV', 'TPV',
       'TSL','TCV', 'TSL']).columns.values.tolist())

In [ ]:
best_gb_model = models[4]
best_gb_model.get_feature_importance(prettified=True)

# Target: TSV

In [ ]:
models, oof_pred = oof_trainer(X=tsl_target_data[tsl_target_data.TSV != 2],
                    y=tsl_target_data[tsl_target_data.TSV != 2].TSV,
                    n_folds = 5,
                   # params=params,
                    del_cols=['TSV',
       'TSL','TCV', 'TSL', 'TPV'],
                    cat_features=cat_features.drop(columns=['TSV', 'TPV',
       'TSL','TCV', 'TSL']).columns.values.tolist())

In [ ]:
best_gb_model = models[0]
best_gb_model.get_feature_importance(prettified=True)

# Target: TCV

In [ ]:
models, oof_pred = oof_trainer(X=tsl_target_data[tsl_target_data.TCV != -3],
                    y=tsl_target_data[tsl_target_data.TCV != -3].TCV,
                    n_folds = 5,
                   # params=params,
                    del_cols=['TSV',
       'TSL','TCV', 'TSL', 'TPV'],
                    cat_features=cat_features.drop(columns=['TSV', 'TPV',
       'TSL','TCV', 'TSL']).columns.values.tolist())

In [ ]:
best_gb_model = models[0]
best_gb_model.get_feature_importance(prettified=True)